In [1]:
from scipy.stats import bernoulli, binom, uniform, norm, poisson, t, ttest_1samp, ttest_ind
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import random

In [2]:
df=pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [4]:
new_df=df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
#df.dropna(axis=1)
new_df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
hyp_mean=0.125
alpha=0.025

# Get rest of data from Sample set
N=len(new_df)
x_bar=len(new_df[new_df['v1']=='spam'])/N
x_std=np.sqrt(hyp_mean*(1-hyp_mean))
dof=N-1
x_stderr=x_std/np.sqrt(N)


# Built t distribution
T_dist=t(dof,hyp_mean,x_stderr)
pval=1-T_dist.cdf(x_bar)
pval

0.02041989487069895

In [6]:
rv = poisson(len(new_df[new_df['v1']=='spam'])/28)

In [7]:
x_stderr

0.0044305075313021

In [8]:
# At least 30 spams a day is 1 - cdf of 29 spams >=30 is same as <29
1-rv.cdf(29)

0.2847229977401552

In [9]:
new_df.loc[random.randint(1,N),['v2']]

v2    Please CALL 08712402779 immediately as there i...
Name: 1251, dtype: object

In [10]:
new_df.loc[5302]

v1                                                  ham
v2    About  &lt;#&gt; bucks. The banks fees are fix...
Name: 5302, dtype: object

In [11]:
# You will be creating a Naive Bayes Text Classification Model to determine if an incoming message is spam or not.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

counts = count_vect.fit_transform(new_df['v2']) 
tfidfs = tfidf_transformer.fit_transform(counts)

X_train, X_test, y_train, y_test = train_test_split(tfidfs, new_df['v1'], test_size=0.33, random_state=42) 

nb = MultinomialNB().fit(X_train, y_train)
predictions = nb.predict(X_test)

In [12]:
type(new_df['v1'])

pandas.core.series.Series

In [13]:
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [14]:
y_true=np.array(y_test)

In [15]:
def NB_model_accuracy(predictions,y_true):
    accurate, false_pos, false_neg, results = 0,0,0,[]
    for i in range(len(predictions)):
        if predictions[i]==y_true[i]:
            accurate+=1
        elif predictions[i]=='spam':
            false_pos +=1
        else:
            false_neg +=1
    accuracy=accurate/(accurate+false_pos+false_neg)
    results.append([accuracy, false_neg, false_pos])
    return results

In [16]:
r=NB_model_accuracy(predictions,y_true)
r

[[0.953779227841218, 85, 0]]

In [17]:
new_predictions = nb.predict(tfidfs)
new_predictions

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [18]:
new_y_true=np.array(new_df['v1'])
new_y_true

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [19]:
NB_model_accuracy(new_predictions,new_y_true)

[[0.9603374012921752, 221, 0]]

In [20]:
sum(new_predictions=='spam')

526

In [21]:
hyp_mean=0.125
alpha=0.025

# Get rest of data from Sample set
N=len(new_predictions)
x_bar=sum(new_predictions=='spam')/N
x_std=np.sqrt(hyp_mean*(1-hyp_mean))
dof=N-1
x_stderr=x_std/np.sqrt(N)


# Built t distribution
T_dist=t(dof,hyp_mean,x_stderr)
pval=1-T_dist.cdf(x_bar)
1-pval

2.760236483823064e-12